<div style="background: black; padding: 10px 250px"><img src="https://www.veldikompetens.se/wp-content/themes/consid/static/icons/VeldiKompetens_Logo_Web_Negative.svg" title="Veldi kompetens" /></div>

<hr><h1><center>Assignment 5a - Generating the dataset</center></h1>

<h3>About the assignment</h3>
<p>In this assignment we will generate the dataset from the simulation we coded last week. You will be changing a minor section of the code to record some data and then we will convert it into a dataframe using pandas which we will then save in a csv format. In 5b we will use some plots and graphs to get a better look and understanding of our data. Worth noting is that there is a lot of data that can be extracted from this simulation. Both the trends of the general status of the society with people being sick, dead, healthy etc which could could be considered one type of valuable data, another could be several different datasets of each person, how long they have been sick and their daily actions. Of course remember that we currently have a very simplified scenario and yet there is so much interesting data to extract, isn't that just brilliant? </p>

<h3> 1. Setup </h3>
<p>Something that has not been introduced until yet is the IPython.display module imported below. It allows us to render pandas dataframes without having to always say something like df at the end of a cell. This functionality will be showcased and used in the last part of this assignment</p>

In [1]:
import numpy as np
import pandas as pd
import random

from IPython.display import display, HTML

<h3> 2. Generating the dataset</h3>
<p>Below is the slightly modified solution proposal from last week. I have made a few artistic changes in terms of how the print statement looks and added a variable for people_susceptible. Also I have rewritten and added another variable called people_immune which increments when either a person is vaccinated, recovered or both (Check the advance days method for this change). As in last weeks assignment you will find instructions for the assignment below all the code. Worth noting is that this is the last you will see of the huge file, next assignment is just about working with a dataframe in pandas, as we did in week 3 but with the addition of introducing plots. </p>

In [15]:
class Person:
    def __init__(self):
        self.recover_prob = 0.2
        self.die_prob = 0.05
        self.init_sick_prob = 0.1
        self.infect_others_prob = 0.05
        self.vaccinated = False
        self.recovered = False
        self.dead = False
        self.sick = False
        
        self.average_meetups = 10
        self.days_sick = 0
        self.init_sick_or_not()
        
    def init_sick_or_not(self):
        """When created each person starts as either sick or healthy"""
        prob = random.random()
        if prob <= self.init_sick_prob:
            # This should yield that about 10% of the people created are sick, the rest healthy
            self.sick = True
        else:
            self.sick = False
    
    def day_passes(self, population, init_scenario):
        """This method describes what happends to each person each day"""
        if self.sick == True:
            self.days_sick += 1
            if init_scenario == False:
                self.infect_others(population)
        
        # Will person recover?
        prob = random.random()
        if prob <= self.recover_prob and self.sick == True:
            # Person has rehabilitated and is now healthy!
            self.sick = False
            self.recovered = True 
        
        # If person is still sick they might die
        prob = random.random()
        if prob <= self.die_prob and self.sick == True:
            self.dead = True
            self.sick = False
            
    def infect_others(self, population):
        person_encounters = random.sample(range(population.size), self.average_meetups)
        for person_id in person_encounters:
            prob = random.random()
            person = population[person_id]
            if prob <= self.infect_others_prob and person.dead == False and person.recovered == False and person.vaccinated == False:
                population[person_id].sick = True  

        
                
        
        

In [16]:
class Village:
    def __init__(self, init_population_size):
        self.population = np.empty(init_population_size, Person)
        self.init_vaccination = 0.2 * init_population_size
        self.daily_vaccination_threshold = 0.04 * init_population_size
        self.vaccination_started = False
        self.generate_inhabitants()
        
        
    def advance_days(self, init_scenario = False):
        """Counts the status of the citizens in the community"""
        people_sick = 0
        people_recovered = 0
        people_dead = 0
        people_vaccinated = 0     
        people_immune = 0
        
        for person in self.population:
            if person.sick == True:
                people_sick += 1
            if person.dead == True:
                people_dead += 1
                
            if person.recovered == True and person.vaccinated == True:
                people_recovered += 1
                people_vaccinated += 1
                people_immune += 1
            elif person.vaccinated == True:
                people_vaccinated += 1
                people_immune += 1
            elif person.recovered == True:
                people_recovered += 1 
                people_immune += 1
            
            person.day_passes(self.population, init_scenario)
        
        people_susceptible = self.population.size - (people_immune + people_dead + people_sick)
        
        if people_sick >= self.init_vaccination and self.vaccination_started == False:
            print(f"Vaccination has started! At the end of the day {people_sick} are sick and the community is on the alert")
            self.vaccination_started = True
            
        if self.vaccination_started == True:
            self.vaccinate_population()
            
        return people_sick, people_recovered, people_dead, people_vaccinated, people_immune, people_susceptible
    
    def generate_inhabitants(self):
        """Generates the population"""
        for i in range(self.population.size):
            self.population[i] = Person()
            
    def vaccinate_population(self):
        people_vaccinated_today = 0
        
        for person in self.population:
            if people_vaccinated_today == self.daily_vaccination_threshold:
                break
            elif person.sick == False and person.dead == False and person.vaccinated == False:
                person.vaccinated = True
                people_vaccinated_today += 1
                
    def gen_df_and_save(self, day_data_array):
        columns = ["Sick", "Recovered", "Deceased", "Vaccinated", "Immune", "Susceptible"]
        df = pd.DataFrame(day_data_array, columns = columns)
        display(df)
        df.to_csv('Ass5a_generated_dataset.csv')
    
    
    def start_simulation(self):
        """This function controls the simulation and what happends in a day"""
        day_data_list = []
        current_day = 0
    
        people_sick, people_recovered, people_dead, people_vaccinated, people_immune, people_susceptible = self.advance_days(init_scenario = True)
        day_data = [people_sick, people_recovered, people_dead, people_vaccinated, people_immune, people_susceptible]
        day_data_list.append(day_data)
        while people_sick != 0:
            print(f"By day {current_day} {people_sick} people are sick, {people_recovered} has recovered and {people_dead} are dead.")
            current_day += 1
            
            people_sick, people_recovered, people_dead, people_vaccinated, people_immune, people_susceptible = self.advance_days()
            day_data = [people_sick, people_recovered, people_dead, people_vaccinated, people_immune, people_susceptible]
            day_data_list.append(day_data)

        # The village is free of the virus, simulation ended
        day_data_array = np.asarray(day_data_list)
        days_sick = [person.days_sick for person in self.population]
        self.gen_df_and_save(day_data_array)
        print("\n-------END OF SIMULATION-------")
        print(f"By day {current_day} {people_sick} people are sick, {people_recovered} has recovered and {people_dead} are dead.")
        print(f"In total {people_vaccinated} people recieved vaccination and {people_susceptible} remain susceptible to the virus.")
        print("--------------")
        print("The village has recovered and the virus is eliminated!")
        print("The longest time an individual was sick is: ", max(days_sick), "days")
        print(day_data_array)
        
        

<h4>2.3 The main function </h4>
<p>The main function is from where all the code is ran and initializes the simulation with a start population size and creates an instance of the village class. If you want to rerun the whole simulation you just need to rerun the function below as it runs everything else.</p>

In [17]:
def main():
    pop_size = 1000
    village = Village(pop_size)
    village.start_simulation()

main()

By day 0 93 people are sick, 0 has recovered and 0 are dead.
By day 1 97 people are sick, 16 has recovered and 2 are dead.
By day 2 133 people are sick, 30 has recovered and 4 are dead.
By day 3 150 people are sick, 58 has recovered and 9 are dead.
By day 4 193 people are sick, 81 has recovered and 16 are dead.
Vaccination has started! At the end of the day 230 are sick and the community is on the alert
By day 5 230 people are sick, 115 has recovered and 24 are dead.
By day 6 250 people are sick, 156 has recovered and 37 are dead.
By day 7 251 people are sick, 208 has recovered and 42 are dead.
By day 8 237 people are sick, 260 has recovered and 50 are dead.
By day 9 221 people are sick, 313 has recovered and 60 are dead.
By day 10 188 people are sick, 361 has recovered and 67 are dead.
By day 11 158 people are sick, 403 has recovered and 76 are dead.
By day 12 148 people are sick, 427 has recovered and 81 are dead.
By day 13 113 people are sick, 464 has recovered and 89 are dead.
By d

,Sick,Recovered,Deceased,Vaccinated,Immune,Susceptible
0,93,0,0,0,0,907
1,97,16,2,0,16,885
2,133,30,4,0,30,833
3,150,58,9,0,58,783
4,193,81,16,0,81,710
5,230,115,24,0,115,631
6,250,156,37,40,191,522
7,251,208,42,80,273,434
8,237,260,50,120,345,368
9,221,313,60,160,415,304



-------END OF SIMULATION-------
By day 36 0 people are sick, 580 has recovered and 115 are dead.
In total 885 people recieved vaccination and 0 remain susceptible to the virus.
--------------
The village has recovered and the virus is eliminated!
The longest time an individual was sick is:  27 days
[[ 93   0   0   0   0 907]
 [ 97  16   2   0  16 885]
 [133  30   4   0  30 833]
 [150  58   9   0  58 783]
 [193  81  16   0  81 710]
 [230 115  24   0 115 631]
 [250 156  37  40 191 522]
 [251 208  42  80 273 434]
 [237 260  50 120 345 368]
 [221 313  60 160 415 304]
 [188 361  67 200 479 266]
 [158 403  76 240 537 229]
 [148 427  81 280 576 195]
 [113 464  89 320 624 174]
 [ 94 488  92 360 660 154]
 [ 76 505  98 400 689 137]
 [ 61 519 103 440 716 120]
 [ 46 536 104 480 744 106]
 [ 38 542 108 520 763  91]
 [ 33 549 109 560 778  80]
 [ 28 556 109 600 797  66]
 [ 24 559 110 640 809  57]
 [ 24 560 111 680 821  44]
 [ 20 562 113 720 833  34]
 [ 13 569 113 760 849  25]
 [ 10 571 114 800 862  1

<h3>3. The assignment </h3>
This assignment is divided into two sections; the first section is about creating a data array which is done in the start_simulation method,the second section is about converting this data array into a dataframe and then saving it in a csv format. Although a lot of data can be extracted from this scenario, we will focus on extracting the day by day progress of the village. The dataset will thus contain a record of each day with the amount of people sick, recovered, deceased, vaccinated, immune and susceptible. Lets start by creating this data array.
<h4>3.1 Generating the data array </h4>
<p>Creating the data array is both a bit tedious but also fairly simple. To generate the data array we will be making use of both numpy and regular python lists. Appending and incrementing sizes of numpy arrays are very expensive in terms of computational cost, thus we will use a regular two-dimensional python list at first, and then convert it into a numpy array.  We do not need to save each day as it kind of comes naturally with the format, since each row represents a day and the very first row represents the first day. </p>
<h5>Instructions</h5>
<p>All of the following will be done in the start_simulation method in the village class above (Working from the top of the method and downwards)</p>
<ul>
    <li style="padding-bottom: 16px;">Create a new local variable at the start of the method called day_data_list and set it to an empty python list (not a numpy array)</li>
    <li style="padding-bottom: 16px;">Just before the while-loop create a variable called day_data which you create as a python list of the values people_sick, people_recovered, people_dead, people_vaccinated, people_immune and people_susceptible (NOTE: The order is important)</li>
    <li style="padding-bottom: 16px;">After the part where you create the day_data variable, append the day_data variable to the day_data_list variable we created at the start of the method, if you want to validate that the the day_data variable corresponds correctly to the print statements generated when you run the program, you can use another print statement to display the list. </li>
        <li style="padding-bottom: 16px;">Once again perform the two steps described above but this time do it inside the while-loop sticking with the same format as how we did it above the while loop</li>
        <li style="padding-bottom: 16px;">Finally, after the while loop create a variable day_data_array which should be a numpy array using the day_data_list variable we created.</li>
</ul>

<p>You now have your data array! Woho! Below you will find the output I get when running the program and the two dimensioanl numpy array that corresponds to the days at the very bottom. I added a print statement at the end of the method where I printed the numpy array. !!Remember!! You will never get the exact same output as me, but ensure that the print statements for each day is in sync with your array. </p>

<h5>Somewhat expected output: </h5>

<pre>By day 0 106 people are sick, 0 has recovered and 0 are dead.
By day 1 98 people are sick, 21 has recovered and 3 are dead.
By day 2 111 people are sick, 46 has recovered and 5 are dead.
By day 3 143 people are sick, 60 has recovered and 7 are dead.
By day 4 157 people are sick, 101 has recovered and 11 are dead.
By day 5 196 people are sick, 128 has recovered and 17 are dead.
Vaccination has started! At the end of the day 209 are sick and the community is on the alert
By day 6 209 people are sick, 164 has recovered and 26 are dead.
By day 7 229 people are sick, 204 has recovered and 35 are dead.
By day 8 244 people are sick, 248 has recovered and 40 are dead.
By day 9 219 people are sick, 308 has recovered and 48 are dead.
By day 10 190 people are sick, 360 has recovered and 58 are dead.
By day 11 173 people are sick, 398 has recovered and 68 are dead.
By day 12 155 people are sick, 430 has recovered and 78 are dead.
By day 13 141 people are sick, 460 has recovered and 81 are dead.
By day 14 126 people are sick, 484 has recovered and 82 are dead.
By day 15 108 people are sick, 505 has recovered and 87 are dead.
By day 16 91 people are sick, 526 has recovered and 90 are dead.
By day 17 73 people are sick, 544 has recovered and 92 are dead.
By day 18 60 people are sick, 557 has recovered and 94 are dead.
By day 19 47 people are sick, 569 has recovered and 96 are dead.
By day 20 43 people are sick, 575 has recovered and 98 are dead.
By day 21 38 people are sick, 578 has recovered and 102 are dead.
By day 22 32 people are sick, 584 has recovered and 103 are dead.
By day 23 24 people are sick, 590 has recovered and 105 are dead.
By day 24 19 people are sick, 595 has recovered and 105 are dead.
By day 25 14 people are sick, 599 has recovered and 106 are dead.
By day 26 9 people are sick, 602 has recovered and 108 are dead.
By day 27 9 people are sick, 602 has recovered and 108 are dead.
By day 28 6 people are sick, 604 has recovered and 109 are dead.
By day 29 5 people are sick, 605 has recovered and 109 are dead.
By day 30 4 people are sick, 606 has recovered and 109 are dead.
By day 31 4 people are sick, 606 has recovered and 109 are dead.
By day 32 3 people are sick, 607 has recovered and 109 are dead.
By day 33 3 people are sick, 607 has recovered and 109 are dead.
By day 34 2 people are sick, 608 has recovered and 109 are dead.
By day 35 2 people are sick, 608 has recovered and 109 are dead.
By day 36 1 people are sick, 608 has recovered and 110 are dead.
By day 37 1 people are sick, 608 has recovered and 110 are dead.
By day 38 1 people are sick, 608 has recovered and 110 are dead.
</pre>
<pre>
-------END OF SIMULATION-------
By day 39 0 people are sick, 609 has recovered and 110 are dead.
In total 890 people recieved vaccination and 0 remain susceptible to the virus.
--------------
The village has recovered and the virus is eliminated!
The longest time an individual was sick is:  32 days
[[106   0   0   0   0 894]
 [ 98  21   3   0  21 878]
 [111  46   5   0  46 838]
 [143  60   7   0  60 790]
 [157 101  11   0 101 731]
 [196 128  17   0 128 659]
 [209 164  26   0 164 601]
 [229 204  35  40 232 504]
 [244 248  40  80 301 415]
 [219 308  48 120 377 356]
 [190 360  58 160 448 304]
 [173 398  68 200 501 258]
 [155 430  78 240 547 220]
 [141 460  81 280 589 189]
 [126 484  82 320 629 163]
 [108 505  87 360 660 145]
 [ 91 526  90 400 694 125]
 [ 73 544  92 440 721 114]
 [ 60 557  94 480 749  97]
 [ 47 569  96 520 772  85]
 [ 43 575  98 560 786  73]
 [ 38 578 102 600 797  63]
 [ 32 584 103 640 811  54]
 [ 24 590 105 680 829  42]
 [ 19 595 105 720 842  34]
 [ 14 599 106 760 858  22]
 [  9 602 108 800 868  15]
 [  9 602 108 840 877   6]
 [  6 604 109 880 884   1]
 [  5 605 109 886 886   0]
 [  4 606 109 887 887   0]
 [  4 606 109 887 887   0]
 [  3 607 109 888 888   0]
 [  3 607 109 888 888   0]
 [  2 608 109 889 889   0]
 [  2 608 109 889 889   0]
 [  1 608 110 889 889   0]
 [  1 608 110 889 889   0]
 [  1 608 110 889 889   0]
 [  0 609 110 890 890   0]]
</pre>

<h4>3.2 Creating the data frame and saving as csv file </h4>
<p>We will now convert our numpy array into a dataframe to visualize it and then also save the dataframe as a csv file. </p>
<h5>Instructions</h5>
<ul>
    <li style="padding-bottom: 16px;">In the start_simulation method just after the days_sick variable and before all the print statements at the end, add a call to a method we will soon create that will be called gen_df_and_save and pass the day_data_array as parameter (remeber you need to say self.ur-method-name to call a method inside the class)</li>
    <li style="padding-bottom: 16px;">Create the method gen_df_and_save. Remember it needs to accept a parameter, call this paramter day_data_array since it corresponds to the variable in the start_simulation method</li>
     <li style="padding-bottom: 16px;">Create a variable columns which is a regular Python list that corresponds to the columns in our data. The variable should thus contain 6 strings ordered as Sick, Recovered, Deceased, Vaccinated, Immune and Susceptible. The order is very important to get correct </li>
    <li style="padding-bottom: 16px;">Create a variable df which is created as a dataframe using pd.dataframe(name-of-ur-array, columns = your-columns-variable)</li>
    <li style="padding-bottom: 16px;">You can now display the dataframe using display(df), display here is a functionality we imported in the setup section of this assignment and you will see the very nice table rendered the next time you run the program. Finally save the dataframe as a csv file and name it Ass5a_generated_dataset </li>
    <li style="padding-bottom: 16px;">Run the program and you should see a lovely dataframe generated and a csv file of the dataframe appear in your folder</li>
</ul>

<h5>Somewhat expected output (here I only display my dataframe and not the other print statements): </h5>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Sick</th>
      <th>Recovered</th>
      <th>Deceased</th>
      <th>Vaccinated</th>
      <th>Immune</th>
      <th>Susceptible</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>106</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>894</td>
    </tr>
    <tr>
      <th>1</th>
      <td>98</td>
      <td>21</td>
      <td>3</td>
      <td>0</td>
      <td>21</td>
      <td>878</td>
    </tr>
    <tr>
      <th>2</th>
      <td>111</td>
      <td>46</td>
      <td>5</td>
      <td>0</td>
      <td>46</td>
      <td>838</td>
    </tr>
    <tr>
      <th>3</th>
      <td>143</td>
      <td>60</td>
      <td>7</td>
      <td>0</td>
      <td>60</td>
      <td>790</td>
    </tr>
    <tr>
      <th>4</th>
      <td>157</td>
      <td>101</td>
      <td>11</td>
      <td>0</td>
      <td>101</td>
      <td>731</td>
    </tr>
    <tr>
      <th>5</th>
      <td>196</td>
      <td>128</td>
      <td>17</td>
      <td>0</td>
      <td>128</td>
      <td>659</td>
    </tr>
    <tr>
      <th>6</th>
      <td>209</td>
      <td>164</td>
      <td>26</td>
      <td>0</td>
      <td>164</td>
      <td>601</td>
    </tr>
    <tr>
      <th>7</th>
      <td>229</td>
      <td>204</td>
      <td>35</td>
      <td>40</td>
      <td>232</td>
      <td>504</td>
    </tr>
    <tr>
      <th>8</th>
      <td>244</td>
      <td>248</td>
      <td>40</td>
      <td>80</td>
      <td>301</td>
      <td>415</td>
    </tr>
    <tr>
      <th>9</th>
      <td>219</td>
      <td>308</td>
      <td>48</td>
      <td>120</td>
      <td>377</td>
      <td>356</td>
    </tr>
    <tr>
      <th>10</th>
      <td>190</td>
      <td>360</td>
      <td>58</td>
      <td>160</td>
      <td>448</td>
      <td>304</td>
    </tr>
    <tr>
      <th>11</th>
      <td>173</td>
      <td>398</td>
      <td>68</td>
      <td>200</td>
      <td>501</td>
      <td>258</td>
    </tr>
    <tr>
      <th>12</th>
      <td>155</td>
      <td>430</td>
      <td>78</td>
      <td>240</td>
      <td>547</td>
      <td>220</td>
    </tr>
    <tr>
      <th>13</th>
      <td>141</td>
      <td>460</td>
      <td>81</td>
      <td>280</td>
      <td>589</td>
      <td>189</td>
    </tr>
    <tr>
      <th>14</th>
      <td>126</td>
      <td>484</td>
      <td>82</td>
      <td>320</td>
      <td>629</td>
      <td>163</td>
    </tr>
    <tr>
      <th>15</th>
      <td>108</td>
      <td>505</td>
      <td>87</td>
      <td>360</td>
      <td>660</td>
      <td>145</td>
    </tr>
    <tr>
      <th>16</th>
      <td>91</td>
      <td>526</td>
      <td>90</td>
      <td>400</td>
      <td>694</td>
      <td>125</td>
    </tr>
    <tr>
      <th>17</th>
      <td>73</td>
      <td>544</td>
      <td>92</td>
      <td>440</td>
      <td>721</td>
      <td>114</td>
    </tr>
    <tr>
      <th>18</th>
      <td>60</td>
      <td>557</td>
      <td>94</td>
      <td>480</td>
      <td>749</td>
      <td>97</td>
    </tr>
    <tr>
      <th>19</th>
      <td>47</td>
      <td>569</td>
      <td>96</td>
      <td>520</td>
      <td>772</td>
      <td>85</td>
    </tr>
    <tr>
      <th>20</th>
      <td>43</td>
      <td>575</td>
      <td>98</td>
      <td>560</td>
      <td>786</td>
      <td>73</td>
    </tr>
    <tr>
      <th>21</th>
      <td>38</td>
      <td>578</td>
      <td>102</td>
      <td>600</td>
      <td>797</td>
      <td>63</td>
    </tr>
    <tr>
      <th>22</th>
      <td>32</td>
      <td>584</td>
      <td>103</td>
      <td>640</td>
      <td>811</td>
      <td>54</td>
    </tr>
    <tr>
      <th>23</th>
      <td>24</td>
      <td>590</td>
      <td>105</td>
      <td>680</td>
      <td>829</td>
      <td>42</td>
    </tr>
    <tr>
      <th>24</th>
      <td>19</td>
      <td>595</td>
      <td>105</td>
      <td>720</td>
      <td>842</td>
      <td>34</td>
    </tr>
    <tr>
      <th>25</th>
      <td>14</td>
      <td>599</td>
      <td>106</td>
      <td>760</td>
      <td>858</td>
      <td>22</td>
    </tr>
    <tr>
      <th>26</th>
      <td>9</td>
      <td>602</td>
      <td>108</td>
      <td>800</td>
      <td>868</td>
      <td>15</td>
    </tr>
    <tr>
      <th>27</th>
      <td>9</td>
      <td>602</td>
      <td>108</td>
      <td>840</td>
      <td>877</td>
      <td>6</td>
    </tr>
    <tr>
      <th>28</th>
      <td>6</td>
      <td>604</td>
      <td>109</td>
      <td>880</td>
      <td>884</td>
      <td>1</td>
    </tr>
    <tr>
      <th>29</th>
      <td>5</td>
      <td>605</td>
      <td>109</td>
      <td>886</td>
      <td>886</td>
      <td>0</td>
    </tr>
    <tr>
      <th>30</th>
      <td>4</td>
      <td>606</td>
      <td>109</td>
      <td>887</td>
      <td>887</td>
      <td>0</td>
    </tr>
    <tr>
      <th>31</th>
      <td>4</td>
      <td>606</td>
      <td>109</td>
      <td>887</td>
      <td>887</td>
      <td>0</td>
    </tr>
    <tr>
      <th>32</th>
      <td>3</td>
      <td>607</td>
      <td>109</td>
      <td>888</td>
      <td>888</td>
      <td>0</td>
    </tr>
    <tr>
      <th>33</th>
      <td>3</td>
      <td>607</td>
      <td>109</td>
      <td>888</td>
      <td>888</td>
      <td>0</td>
    </tr>
    <tr>
      <th>34</th>
      <td>2</td>
      <td>608</td>
      <td>109</td>
      <td>889</td>
      <td>889</td>
      <td>0</td>
    </tr>
    <tr>
      <th>35</th>
      <td>2</td>
      <td>608</td>
      <td>109</td>
      <td>889</td>
      <td>889</td>
      <td>0</td>
    </tr>
    <tr>
      <th>36</th>
      <td>1</td>
      <td>608</td>
      <td>110</td>
      <td>889</td>
      <td>889</td>
      <td>0</td>
    </tr>
    <tr>
      <th>37</th>
      <td>1</td>
      <td>608</td>
      <td>110</td>
      <td>889</td>
      <td>889</td>
      <td>0</td>
    </tr>
    <tr>
      <th>38</th>
      <td>1</td>
      <td>608</td>
      <td>110</td>
      <td>889</td>
      <td>889</td>
      <td>0</td>
    </tr>
    <tr>
      <th>39</th>
      <td>0</td>
      <td>609</td>
      <td>110</td>
      <td>890</td>
      <td>890</td>
      <td>0</td>
    </tr>
  </tbody>
</table>